In [ ]:
### Preparation of packages

!pip install selfcheckgpt
!pip install -U spacy
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_core_sci_lg-0.5.3.tar.gz


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00
  Created wheel for selfcheckgpt: filename=selfcheckgpt-0.1.4-py3-none-any.whl size=12240 sha256=68c703d857f9ac809e1e519fbc6a31b4e7de5b2cdf123bf823ab444d040d5ddf
  Stored in directory: /root/.cache/pip/wheels/dc/a6/76/107247d4924cb23f6e1b1987413b2a191979767e1e2c1d5751
Successfully built selfcheckgpt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 932.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 7.9 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Created wheel for nmslib: filename=nmslib-2.1.1-cp310-cp310-linux_x86_64.whl size=13578648 sha256=2ade11d0cf19a46c76666b13d6565a088a77dfc2e34dcc9f00e958a5bc17be3f
  Stored in directory: /root/.cache/pip/wheels/21/1a/5d/4cc754a5b1a88405cad184b76f823897a63a8d19afcd4b9314
Successfully built nmslib
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
  Attempting uninstall: spacy
    Found exist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.2/531.2 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for en-core-sci-lg: filename=en_core_sci_lg-0.5.3-py3-none-any.whl size=531476666 sha256=1ac274f0ddefbd6cec9f54fcd395caa21231b476c2e07aabae231781e5f06c90
  Stored in directory: /root/.cache/pip/wheels/98/42/c4/404d162d1ffdbae4329b3bcfdf2ed2af5afcb9fea4bd274736
Successfully built en-core-sci-lg


In [ ]:


import re
import os
import csv
import json
from typing import List
import yaml

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm, trange


In [ ]:
from nltk.translate.bleu_score import corpus_bleu
import requests
import xml.etree.ElementTree as ET
from nltk.translate.bleu_score import corpus_bleu

import nltk
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
nltk.download('punkt')  # Download the 'punkt' resource
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
### import python packages

import nltk
import spacy
import scispacy
import numpy as np
from selfcheckgpt.modeling_selfcheck import SelfCheckBERTScore
from nltk.translate.meteor_score import single_meteor_score

# download wordnet
nltk.download('wordnet')

# Use SpaCy to extract clinical related entities
nlp = spacy.load("en_core_sci_lg")


[nltk_data] Downloading package wordnet to /root/nltk_data...
/usr/local/lib/python3.10/dist-packages/spacy/language.py:2141: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [ ]:
import pickle
with open("/content/drive/MyDrive/Capstone project/Ethics/llama_stats.pkl", 'rb') as file:
    data = pickle.load(file)

In [ ]:
clinical_note = data[0]['clinical_note']
summary = data[0]['summary']
reference_summary = data[0]['reference_summary']
m_score = data[0]['meteor_score']
r1 = data[0]['rouge1_score']
r2 = data[0]['rouge2_score']
rl = data[0]['rougeL_score']
bl = data[0]['bleu_score']

print("Clinical Note:", clinical_note)
print("Summary:", summary)
print("Reference Summary:", reference_summary)
print("meteor_score", m_score)
print("rouge1_score", r1)
print("rouge2_score", r2)
print("rougeL_score", rl)
print("bleu_score", bl)

Clinical Note: ['78 M transferred to nursing home for rehab after CABG. Reportedly readmitted with a small NQWMI. Yesterday, he was noted to have a melanotic stool and then today he had approximately 9 loose BM w/ some melena and some frank blood just prior to transfer, unclear quantity.']
Summary: ['```\nPlease note that I am not a doctor or medical professional but will try my best to provide you information from this situation based on what I know of it. It seems like there might be two different issues at hand here - one being a possible bleeding issue in the GI tract which may have led to a hemorrhagic shock (if there were any significant amount of blood loss) as well as potential signs of dehydration due to vomiting. In addition, it appears that your patient has been recently discharged from the hospital following surgery so it&#x27;s likely they are also dealing with other post-operative concerns such as pain management etc.. If we focus specifically on their bowel movements tho

In [ ]:
import pandas as pd

df = pd.read_excel('/content/drive/MyDrive/Capstone project/Ethics/Outputs/llama2_output.xlsx')
df.head()

,articles,correct_summary,model_generated,meteor_scores
0,"during the last years , the self - assembly of...","during the last years , self - assembled organ...",Sure! Here&#x27;s a consolidated summary of th...,0.293046
1,acute pancreatitis is an inflammatory intra - ...,aim . \n acute pancreatitis is an inflammatory...,"Great, thank you! Based on these four summarie...",0.139674
2,pathological gambling is defined in the curren...,abstractthe incidence of pathological gambling...,Sure! Here&#x27;s a consolidated summary of th...,0.318883
3,exacerbations of copd are associated with acce...,purposepulmonary hypertension and exercise - i...,Here&#x27;s a consolidated summary of the four...,0.326362
4,peri - implantitis is inflammation of the peri...,an 810-nm diode laser was used to non - surgic...,Sure! Here&#x27;s a consolidated summary of th...,0.190171


In [ ]:
model_generated = df['model_generated'].tolist()
correct_summary = df['correct_summary'].tolist()
articles = df['articles'].tolist()
print(len(model_generated))
print(len(correct_summary))
print(len(articles))

100
100
100


In [ ]:
### SelfCheckGPT Score

# pre-define this selfcheck_bertscore function before all the iteration code
selfcheck_bertscore = SelfCheckBERTScore(rescale_with_baseline=True)

# We need to change the source code to extract sentence for sampled_passages
def SelfCheckGPT(pred,label):
    sent_scores_bertscore = selfcheck_bertscore.predict(
      sentences = [sent.text.strip() for sent in nlp(label).sents if len(sent) > 3],
      sampled_passages = [pred.strip()],
    )
    return np.mean(sent_scores_bertscore)

SelfCheck-BERTScore initialized


In [ ]:
### Meteor Score for Hallucination

# use 1 minus meteor score to estimate hallucination
def Meteor_Hallucination(pred,label):
    doc1, doc2 = nlp(pred), nlp(label)
    x, y = [str(d) for d in doc1.ents], [str(d) for d in doc2.ents]
    return 1 - single_meteor_score(y,x)

In [ ]:
doc1, doc2 = nlp(model_generated[0]), nlp(correct_summary[0])
x, y = [str(d) for d in doc1.ents], [str(d) for d in doc2.ents]

['years',
 'self',
 'assembled',
 'organic',
 'electrical',
 'optical applications',
 'phenylenes',
 'deposited',
 'muscovite mica',
 'material',
 'combination',
 'needlelike structures',
 'macroscopic',
 'emission',
 'waveguiding',
 'lasing',
 'resulting anisotropy',
 'induced',
 'dipole',
 'muscovite mica',
 'substrate',
 'experimental',
 'theoretical approach',
 'alternative',
 'growth model',
 'molecular adsorption',
 'sheet',
 'silicates',
 'moleculesurface',
 'interactions',
 'comprehensive',
 'comparison',
 'experiments',
 'simulations',
 'geometrical changes',
 'substrate',
 'surface',
 'molecule',
 'molecular adsorption geometries',
 'needle',
 'growth model']

In [ ]:
len(doc1)

353

In [ ]:
doc1

Sure! Here&#x27;s a consolidated summary of the articles based on your requests:
In this study, researchers investigate the creation of organized nanostructures with potential applications in device technology. They discover a method to create parallel-aligned nanofibers emitting blue light by aligning molecules on muscovite mica. Understanding the growth mechanism is essential for selecting the best molecules for creating these structures. Using X-ray diffraction and atomic force microscopy, the study shows that the surface geometry of muscovite mica influences the growth behavior of the nanofibers. On phlogopite mica, nanoneedles with a higher degree of symmetry are observed compared to muscovite mica. Further analysis reveals that the lattice parameters of the growing crystallites are affected by the surface geometry of the substrate.
A subsequent study examines the structural properties of hydrographitically grown 6T nanofibers using X-ray diffraction. The results indicate that the

In [ ]:
doc1.ents

(Sure!,
 articles,
 requests,
 study,
 researchers,
 investigate,
 creation,
 organized,
 nanostructures,
 device technology,
 method,
 parallel-aligned,
 nanofibers,
 molecules,
 muscovite mica,
 growth mechanism,
 molecules,
 structures,
 X-ray diffraction,
 atomic force microscopy,
 study,
 surface geometry,
 muscovite mica,
 influences,
 growth behavior,
 nanofibers,
 phlogopite mica,
 nanoneedles,
 degree,
 symmetry,
 muscovite mica,
 analysis,
 lattice,
 parameters,
 crystallites,
 surface geometry,
 substrate,
 study,
 structural,
 properties,
 hydrographitically grown,
 T nanofibers,
 X-ray diffraction,
 results,
 T molecules,
 crystal,
 orientation,
 planes,
 substrate,
 surface,
 study,
 long molecular axis,
 T molecules,
 density,
 molecules,
 surface,
 increases,
 molecules,
 changes,
 orientations,
 Needle,
 study,
 influence,
 surface,
 corrugation,
 adsorption,
 organic molecules,
 sheet,
 silicate,
 substrates,
 investigation,
 surface,
 corrugation,
 azimuthal,
 orient

In [ ]:
model_generated[0]

'Sure! Here&#x27;s a consolidated summary of the articles based on your requests:\nIn this study, researchers investigate the creation of organized nanostructures with potential applications in device technology. They discover a method to create parallel-aligned nanofibers emitting blue light by aligning molecules on muscovite mica. Understanding the growth mechanism is essential for selecting the best molecules for creating these structures. Using X-ray diffraction and atomic force microscopy, the study shows that the surface geometry of muscovite mica influences the growth behavior of the nanofibers. On phlogopite mica, nanoneedles with a higher degree of symmetry are observed compared to muscovite mica. Further analysis reveals that the lattice parameters of the growing crystallites are affected by the surface geometry of the substrate.\nA subsequent study examines the structural properties of hydrographitically grown 6T nanofibers using X-ray diffraction. The results indicate that 

In [ ]:
# Convert SpaCy Doc object to a list of strings (tokens)
ref_tokens = [token.text for token in doc1]
hyp_tokens = model_generated[0].split()  # assuming this is already a string

# Compute BLEU score
bleu_score = corpus_bleu([[ref_tokens]], [hyp_tokens])

bleu_score

0.705246512013589

In [ ]:
doc2

during the last years , self - assembled organic nanostructures have been recognized as a proper fundament for several electrical and optical applications . in particular , phenylenes deposited on muscovite mica have turned out to be an outstanding material combination . 
 they tend to align parallel to each other forming needlelike structures . in that way 
 , they provide the key for macroscopic highly polarized emission , waveguiding , and lasing . 
 the resulting anisotropy has been interpreted so far by an induced dipole originating from the muscovite mica substrate . 
 based on a combined experimental and theoretical approach , we present an alternative growth model being able to explain molecular adsorption on sheet silicates in terms of moleculesurface interactions only . by a comprehensive comparison between experiments and simulations , 
 we demonstrate that geometrical changes in the substrate surface or molecule lead to different molecular adsorption geometries and needle d

In [ ]:
### Hallucination Score

# 1 - precision
# 1 - TP / (TP+FP)
# TP: common entities between generated text and label text
# use this to estimate hallucination

def Hallucination_Score(pred,label):
    doc1, doc2 = nlp(pred), nlp(label)
    x, y = [str(d) for d in doc1.ents], [str(d) for d in doc2.ents]
    if (len(x) == 0):
        return 1
    else:
        return 1 - len(list(set(list(x))&set(list(y))))/len(set(list(x)))

In [ ]:
SelfCheckGPT(model_generated[0], correct_summary[0])

0.8570692988303461

In [ ]:
Meteor_Hallucination(model_generated[0], correct_summary[0])

0.8945147679324894

In [ ]:
Hallucination_Score(model_generated[0], correct_summary[0])

0.9466666666666667

In [ ]:
self_c = []
met_c = []
hall_c = []
for i in range(100):
  a = SelfCheckGPT(model_generated[i], correct_summary[i])
  self_c.append(a)
  b = Meteor_Hallucination(model_generated[i], correct_summary[i])
  met_c.append(b)
  c = Hallucination_Score(model_generated[i], correct_summary[i])
  hall_c.append(c)

In [ ]:
self_a = []
met_a = []
hall_a = []
for i in range(100):
  a = SelfCheckGPT(model_generated[i], articles[i])
  self_a.append(a)
  b = Meteor_Hallucination(model_generated[i], articles[i])
  met_a.append(b)
  c = Hallucination_Score(model_generated[i], articles[i])
  hall_a.append(c)

In [ ]:
data = {
    'articles': articles,
    'correct_summary': correct_summary,
    'model_generated': model_generated,
    'SelfCheckGPT (correct and generated)': self_c,
    'Meteor (correct and generated)': met_c,
    'Hallucination (correct and generated)': hall_c,
    'SelfCheckGPT (article and generated)': self_a,
    'Meteor (article and generated)': met_a,
    'Hallucination (article and generated)': hall_a
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Exporting the DataFrame to an Excel file
df.to_excel('Hallucination metrics.xlsx', index=False)

In [ ]:
SelfCheckGPT(data[0]['summary'][0], data[0]['reference_summary'])

NameError: ignored

In [ ]:
Meteor_Hallucination(data[0]['summary'][0], data[0]['reference_summary'])

1.0

In [ ]:
Hallucination_Score(data[0]['summary'][0], data[0]['reference_summary'])

1.0

In [ ]:
for i in range(30):
  a = SelfCheckGPT(data[i]['summary'][0], data[i]['reference_summary'])
  b = Meteor_Hallucination(data[i]['summary'][0], data[i]['reference_summary'])
  c = Hallucination_Score(data[i]['summary'][0], data[i]['reference_summary'])


In [ ]:
self_check = []
met_hal = []
hal_score = []
for i in range(30):
  data[i]['SelfCheckGPT'] = SelfCheckGPT(data[i]['summary'][0], data[i]['reference_summary'])
  self_check.append(data[i]['SelfCheckGPT'])
  data[i]['Meteor_Hallucination'] = Meteor_Hallucination(data[i]['summary'][0], data[i]['reference_summary'])
  met_hal.append(data[i]['Meteor_Hallucination'])
  data[i]['Hallucination_Score'] = Hallucination_Score(data[i]['summary'][0], data[i]['reference_summary'])
  hal_score.append(data[i]['Hallucination_Score'])

In [ ]:
self_check

[1.0086725987493992,
 0.6306819319725037,
 0.8473836311272213,
 0.9689824916422367,
 0.8286270946264267,
 0.8634861260652542,
 0.8167059421539307,
 1.0298785604536533,
 0.9480634711217135,
 0.8298783525824547,
 0.7511541396379471,
 1.2414914419253666,
 0.8612864588697752,
 0.7805741131305695,
 0.8524574087932706,
 1.029216205701232,
 0.9649914093315601,
 0.8039848729968071,
 0.9501546304672956,
 0.9638310937443748,
 0.7770094759762287,
 1.1306889951229095,
 1.379640281200409,
 0.8544065058231354,
 1.0599774979054928,
 0.8988504702380548,
 0.7727480456233025,
 0.9116519466042519,
 0.8535107132047415,
 0.9406684376299381]

In [ ]:
met_hal

[1.0,
 0.9117647058823529,
 0.9800796812749004,
 1.0,
 1.0,
 0.9673202614379085,
 0.9438202247191011,
 0.954954954954955,
 1.0,
 1.0,
 0.8818897637795275,
 1.0,
 0.96,
 1.0,
 1.0,
 1.0,
 0.9621212121212122,
 0.9148936170212766,
 0.8838383838383839,
 1.0,
 0.9056603773584906,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8792207792207792,
 0.9576271186440678,
 1.0,
 0.8196428571428571,
 1.0]

In [ ]:
hal_score

[1.0,
 0.9752066115702479,
 1.0,
 1.0,
 1.0,
 0.9777777777777777,
 0.9411764705882353,
 0.9166666666666666,
 1.0,
 1.0,
 0.9333333333333333,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9571428571428572,
 0.9090909090909091,
 1.0,
 0.9387755102040817,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9347826086956522,
 0.9803921568627451,
 1.0,
 0.9487179487179487,
 1.0]

In [ ]:
with open("/content/drive/MyDrive/Capstone project/Ethics/llama_final_stats.pkl", "wb") as file:
    pickle.dump(data, file)